# Deep-MINE Framework

## 1. Importing libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np

import json

## 2. Loading data

### 2.1 Reviews

In [58]:
# load json
reviews = []
with open('/content/drive/MyDrive/Mestrado/Prime_Pantry.json') as f:
    for line in f:
        reviews.append(json.loads(line))

# create dataframe
df_reviews = pd.DataFrame(reviews)
print(df_reviews.shape)
df_reviews.head()

(471614, 12)


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style
0,5.0,True,"12 14, 2014",A1NKJW0TNRVS7O,B0000DIWNZ,Tamara M.,Good clinging,Clings well,1418515200,NaN,NaN,NaN
1,4.0,True,"11 20, 2014",A2L6X37E8TFTCC,B0000DIWNZ,Amazon Customer,Fantastic buy and a good plastic wrap. Even t...,Saran could use more Plus to Cling better.,1416441600,NaN,NaN,NaN
2,4.0,True,"10 11, 2014",A2WPR4W6V48121,B0000DIWNZ,noname,ok,Four Stars,1412985600,NaN,NaN,NaN
3,3.0,False,"09 1, 2014",A27EE7X7L29UMU,B0000DIWNZ,ZapNZs,Saran Cling Plus is kind of like most of the C...,"The wrap is fantastic, but the dispensing, cut...",1409529600,4,NaN,NaN
4,4.0,True,"08 10, 2014",A1OWT4YZGB5GV9,B0000DIWNZ,Amy Rogers,This is my go to plastic wrap so there isn't m...,has been doing it's job for years,1407628800,NaN,NaN,NaN


In [59]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471614 entries, 0 to 471613
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         471614 non-null  float64
 1   verified        471614 non-null  bool   
 2   reviewTime      471614 non-null  object 
 3   reviewerID      471614 non-null  object 
 4   asin            471614 non-null  object 
 5   reviewerName    471578 non-null  object 
 6   reviewText      471233 non-null  object 
 7   summary         471473 non-null  object 
 8   unixReviewTime  471614 non-null  int64  
 9   vote            41716 non-null   object 
 10  image           3568 non-null    object 
 11  style           6933 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 40.0+ MB


In [61]:
# data cleaning
df_reviews = df_reviews.dropna(subset=['reviewerID', 'asin', 'reviewText'])
df_reviews = df_reviews[['reviewerID', 'asin', 'reviewText']]
print(df_reviews.shape)
df_reviews.head()

(471233, 3)


,reviewerID,asin,reviewText
0,A1NKJW0TNRVS7O,B0000DIWNZ,Good clinging
1,A2L6X37E8TFTCC,B0000DIWNZ,Fantastic buy and a good plastic wrap. Even t...
2,A2WPR4W6V48121,B0000DIWNZ,ok
3,A27EE7X7L29UMU,B0000DIWNZ,Saran Cling Plus is kind of like most of the C...
4,A1OWT4YZGB5GV9,B0000DIWNZ,This is my go to plastic wrap so there isn't m...


In [62]:
df_reviews['reviewerID'].nunique()

247504

In [63]:
df_reviews['asin'].nunique()

10814

In [64]:
df_reviews.shape

(471233, 3)

In [65]:
df_reviews.isnull().sum()

,0
reviewerID,0
asin,0
reviewText,0


### 2.2 Products

In [49]:
# load json
products = []
with open('/content/drive/MyDrive/Mestrado/meta_Prime_Pantry.json') as f:
    for line in f:
        products.append(json.loads(line))

# create dataframe
df_products = pd.DataFrame(products)
print(df_products.shape)
df_products.head(2)

(10813, 19)


,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Sink your sweet tooth into MILK DUDS Candya d...,,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",,Milk Duds,[],[],[],"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",,,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,[],,[Sink your sweet tooth into MILK DUDS Candya d...,,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",,Milk Duds,[],[],[],"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",,,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [50]:
# replace empty lists with null values
df_products = df_products.map(lambda x: np.nan if len(x) == 0 else x)
df_products.head(2)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,NaN,NaN,[Sink your sweet tooth into MILK DUDS Candya d...,NaN,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",NaN,Milk Duds,NaN,NaN,NaN,"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",NaN,NaN,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,NaN,NaN,[Sink your sweet tooth into MILK DUDS Candya d...,NaN,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",NaN,Milk Duds,NaN,NaN,NaN,"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",NaN,NaN,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [51]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10813 entries, 0 to 10812
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   category         0 non-null      float64
 1   tech1            0 non-null      float64
 2   description      10715 non-null  object 
 3   fit              0 non-null      float64
 4   title            10813 non-null  object 
 5   also_buy         4059 non-null   object 
 6   tech2            0 non-null      float64
 7   brand            10810 non-null  object 
 8   feature          1036 non-null   object 
 9   rank             4876 non-null   object 
 10  also_view        5978 non-null   object 
 11  details          10789 non-null  object 
 12  main_cat         10813 non-null  object 
 13  similar_item     0 non-null      float64
 14  date             0 non-null      float64
 15  price            6750 non-null   object 
 16  asin             10813 non-null  object 
 17  imageURL    

In [52]:
all_products = df_products['asin'].unique()
all_products.shape

(10812,)

In [53]:
# data cleaning
df_products = df_products.dropna(subset=['description', 'asin', 'imageURL', 'imageURLHighRes'])
df_products = df_products[['description', 'asin', 'imageURL', 'imageURLHighRes']]
print(df_products.shape)
df_products.head(2)

(8942, 4)


,description,asin,imageURL,imageURLHighRes
0,[Sink your sweet tooth into MILK DUDS Candya d...,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,[Sink your sweet tooth into MILK DUDS Candya d...,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [67]:
df_products.isnull().sum()

,0
description,0
asin,0
imageURL,0
imageURLHighRes,0


In [54]:
# get number of unique products
df_products['asin'].nunique()

8941

In [55]:
# remove the duplicate one
df_products = df_products.drop_duplicates(subset=['asin'])
df_products.shape

(8941, 4)

In [41]:
missing_products = list(set(all_products) - set(df_products['asin']))
len(missing_products)

1871

In [66]:
df_reviews['asin'].isin(missing_products).sum()

np.int64(22706)

In [17]:
df_reviews.merge(df_products, on='asin')

,reviewerID,asin,reviewText,description,imageURL,imageURLHighRes
0,A1NKJW0TNRVS7O,B0000DIWNZ,Good clinging,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,A2L6X37E8TFTCC,B0000DIWNZ,Fantastic buy and a good plastic wrap. Even t...,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2,A2WPR4W6V48121,B0000DIWNZ,ok,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,A27EE7X7L29UMU,B0000DIWNZ,Saran Cling Plus is kind of like most of the C...,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,A1OWT4YZGB5GV9,B0000DIWNZ,This is my go to plastic wrap so there isn't m...,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
...,...,...,...,...,...,...
448462,A2WN3971ONAUJF,B01HI76XS0,I like the taste and convenience of eating the...,[These bars are where our journey started and ...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
448463,AE4FU8QRB3KXA,B01HI76XS0,These are delicious and healthy snacks! I wit...,[These bars are where our journey started and ...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
448464,A3PWAOKOZ8NNGP,B01HI76XS0,Favorite kind bar flavor,[These bars are where our journey started and ...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
448465,A36MOFABIPIPGM,B01HI76XS0,Taste not to be believed. Buy a box for my off...,[These bars are where our journey started and ...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [ ]:
df_reviews.merge(df_products, on='asin')['reviewerID'].value_counts()

,count
reviewerID,
AMMNGUJK4HQJ5,195
A35Q0RBM3YNQNF,175
AKPG8VQBS0MWR,143
A13J2PGKNMJG1K,143
AXK37UZY8UPYP,139
...,...
A3CZPYMZUVA4RD,1
A3QM69AUBKPFVN,1
A39BG3UYXESFOF,1


In [ ]:
df_reviews.merge(df_products, on='asin')['reviewerID'].nunique()

236741

Matrix -> reviewerID (236,741) x asin (8,941)

Feedbacks: 471,233

## 3. Models

In [9]:
# https://discuss.pytorch.org/t/how-to-share-weights-between-two-layers/55541/2
# https://www.kaggle.com/code/ignazio/autoencoder-for-text-in-pytorch


def init_weights():
    pass


class ImageAutoEncoder(nn.Module):

    def __init__(self, tie_weights=True):
        super().__init__()

        # encoder
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.fc3 = nn.Linear(25600, 100, bias=True)

        # decoder
        self.fc4 = nn.Linear(100, 25600, bias=True)
        self.conv5 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1)

        # utils
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.relu = nn.ReLU()

        # share encoder decoder weight matrices
        if tie_weights:
            self._tie_weights()

    def _tie_weights(self):
        self.fc4.weight.data = self.fc3.weight.data.transpose(0,1)
        self.conv5.weight.data = self.conv2.weight.data.transpose(0,1)
        self.conv6.weight.data = self.conv1.weight.data.transpose(0,1)

    def forward(self, x):
        # encoder
        h = self.relu(self.conv1(x))
        h = self.relu(self.conv2(h))
        h = self.pool(h)
        h = self.fc3(h.reshape(-1, 25600))
        print(h.shape)

        # decoder
        h = self.fc4(h).T
        h = h.reshape(-1, 64, 20, 20)
        h = self.upsample(h)
        h = self.conv5(h)
        x_hat = self.conv6(h)
        return x_hat

    def encode(self, x):
        # encoder
        h = self.relu(self.conv1(x))
        h = self.relu(self.conv2(h))
        h = self.pool(h)
        h = self.fc3(h.reshape(-1, 25600))
        return h


class TextAutoEncoder(nn.Module):

    def __init__(self, tie_weights=True):
        super().__init__()

        # encoder
        self.fc1 = nn.Linear(32, 400, bias=True)
        self.fc2 = nn.Linear(400, 100, bias=True)

        # decoder
        self.fc3 = nn.Linear(100, 400, bias=True)
        self.fc4 = nn.Linear(400, 32, bias=True)

        # utils
        self.relu = nn.ReLU()

        # share encoder decoder weight matrices
        if tie_weights:
            self._tie_weights()

    def _tie_weights(self):
        self.fc3.weight.data = self.fc2.weight.data.transpose(0,1)
        self.fc4.weight.data = self.fc1.weight.data.transpose(0,1)

    def forward(self, x):
        # encoder
        h = self.relu(self.fc1(x))
        h = self.fc2(h)
        print(h.shape)

        # decoder
        h = self.relu(self.fc3(h))
        x_hat = self.fc4(h)
        return x_hat

    def encode(self, x):
        # encoder
        h = self.relu(self.fc1(x))
        h = self.fc2(h)
        return h


model = TextAutoEncoder()
print(sum(p.numel() for p in model.parameters()))
print(model)

x = torch.randn(1, 32)
print(x.shape)
model(x).shape

106532
TextAutoEncoder(
  (fc1): Linear(in_features=32, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=400, bias=True)
  (fc4): Linear(in_features=400, out_features=32, bias=True)
  (relu): ReLU()
)
torch.Size([1, 32])
torch.Size([1, 100])


torch.Size([1, 32])

In [ ]:
model = ImageAutoEncoder()
print(sum(p.numel() for p in model.parameters()))
print(model)

x = torch.randn(1, 3, 40, 40)
print(x.shape)
model(x).shape

5223079
ImageAutoEncoder(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc3): Linear(in_features=25600, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=25600, bias=True)
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (upsample): Upsample(scale_factor=2.0, mode='bilinear')
  (relu): ReLU()
)
torch.Size([1, 3, 40, 40])
torch.Size([1, 100])


torch.Size([1, 3, 40, 40])

In [ ]:
# hyparameters
lambda_m = 1/40
lambda_d = 1/32
lambda_r = 1/32

loss = lambda_m/2*mse(x_hat-x) + lambda_w/2

Perhaps check these references:

- https://github.com/sh0416/bpr/blob/master/train.py
- https://github.com/recommenders-team/recommenders/blob/main/examples/02_model_collaborative_filtering/cornac_bpr_deep_dive.ipynb
- https://medium.com/data-science/recommender-system-bayesian-personalized-ranking-from-implicit-feedback-78684bfcddf6
- https://github.com/PreferredAI/cornac/blob/93058c04a15348de60b5190bda90a82dafa9d8b6/cornac/models/vbpr/recom_vbpr.py#L249

**BPR: Bayesian Personalized Ranking**

![](https://seunghan96.github.io/assets/img/recsys/24-4.png)

**VBPR: Visual Bayesian Personalized Ranking**

- Paper: https://arxiv.org/pdf/1510.01784
- Github: https://github.com/aaossa/VBPR-PyTorch
- Github: https://github.com/arogers1/VBPR

![](https://raw.githubusercontent.com/aaossa/VBPR-PyTorch/main/vbpr.png?raw=True)

![](https://velog.velcdn.com/images/hyxxnii/post/732ba898-f3bb-4814-870e-0e49b7e13194/image.png)

## VBPR

In [118]:
n_users = 200
n_items = 500

dim_gamma = 4
dim_theta = 4

input_features = torch.randn(n_items, 4096)

In [119]:
# Image features
features = nn.Embedding.from_pretrained(
    input_features, freeze=True
)  # type: ignore

# Latent factors (gamma)
gamma_users = nn.Embedding(n_users, dim_gamma)
gamma_items = nn.Embedding(n_items, dim_gamma)

# Visual factors (theta)
theta_users = nn.Embedding(n_users, dim_theta)
embedding = nn.Embedding(input_features.size(1), dim_theta)

# Biases (beta)
# beta_users = nn.Embedding(n_users, 1)
beta_items = nn.Embedding(n_items, 1)
visual_bias = nn.Embedding(input_features.size(1), 1)

In [139]:
ui = torch.LongTensor([3])
pi = torch.LongTensor([101])
ni = torch.LongTensor([53])

In [186]:
# User
ui_latent_factors = gamma_users(ui)  # Latent factors of user u
ui_visual_factors = theta_users(ui)  # Visual factors of user u

# Items
# Positive
pi_bias = beta_items(pi)  # Pos. item bias
pi_latent_factors = gamma_items(pi)  # Pos. item visual factors
pi_features = features(pi)  # Pos. item visual features
# Negative
ni_bias = beta_items(ni)  # Neg. item bias
ni_latent_factors = gamma_items(ni)  # Neg. item visual factors
ni_features = features(ni)  # Neg. item visual features

# Precompute differences
diff_features = pi_features - ni_features
diff_latent_factors = pi_latent_factors - ni_latent_factors

# x_uij
x_uij = (
    pi_bias
    - ni_bias
    + (ui_latent_factors * diff_latent_factors).sum(dim=1).unsqueeze(-1)
    + (ui_visual_factors * diff_features.mm(embedding.weight))
    .sum(dim=1)
    .unsqueeze(-1)
    + diff_features.mm(visual_bias.weight)
)
x_uij

tensor([[77.2645]], grad_fn=<AddBackward0>)

In [187]:
outputs = x_uij.unsqueeze(0)
loss = -torch.nn.functional.logsigmoid(outputs).sum()
loss

tensor(2.7826e-34, grad_fn=<NegBackward0>)

In [188]:
x_ui = (ui_latent_factors @ pi_latent_factors.T) + (ui_visual_factors @ embedding.weight.T @ pi_features.T) + pi_features @ visual_bias.weight
x_ui

tensor([[59.6143]], grad_fn=<AddBackward0>)

In [189]:
x_uj = (ui_latent_factors @ ni_latent_factors.T) + (ui_visual_factors @ embedding.weight.T @ ni_features.T) + ni_features @ visual_bias.weight
x_uj

tensor([[-19.3346]], grad_fn=<AddBackward0>)

In [190]:
x_ui - x_uj

tensor([[78.9489]], grad_fn=<SubBackward0>)

In [184]:
-torch.nn.functional.logsigmoid((x_ui - x_uj).unsqueeze(0)).sum()

tensor(5.1631e-35, grad_fn=<NegBackward0>)

## Deep-MINE

In [2]:
n_users = 200
n_items = 500

dim_gamma = 4
dim_theta = 4

input_features = torch.randn(n_items, 4096)

In [38]:
# user perception factors
u_users = nn.Embedding(n_users, 20)
theta_users = nn.Embedding(n_users, 20)

# hidden information factor
v_items = nn.Embedding(n_items, 20)

# cognition factors
a_1 = nn.Embedding(n_users, 1)
a_2 = nn.Embedding(n_users, 1)
a_3 = nn.Embedding(n_users, 1)

# embedding layer of information integration
embeddings = nn.Embedding(300, 20)

# biases
alpha_users = nn.Embedding(n_users, 1)
beta_items = nn.Embedding(n_items, 1)

In [16]:
images_autoencoder = ImageAutoEncoder()
descriptions_autoencoder = TextAutoEncoder()
reviews_autoencoder = TextAutoEncoder()

In [20]:
user = torch.LongTensor([4])
p_item = torch.LongTensor([50])
n_item = torch.LongTensor([98])

In [23]:
# information representation
m3 = images_autoencoder.encode(torch.randn(1, 3, 40, 40))
d2 = descriptions_autoencoder.encode(torch.randn(1, 32))
r2 = reviews_autoencoder.encode(torch.randn(1, 32))

In [24]:
# user's cognitive styles
ai1 = a_1(user)
ai2 = a_2(user)
ai3 = a_3(user)

In [51]:
# information integration
f_c = torch.cat([ai1.mm(m3), ai2.mm(d2), ai3.mm(r2)], dim=-1)
f_j = f_c.mm(embeddings.weight)
f_j.shape

torch.Size([1, 20])

In [55]:
# user preference
ui_latent_factor = u_users(user)
ui_content_factor = theta_users(user)

# hidden information
vj_hidden_info = v_items(p_item)

# biases
ui_bias = alpha_users(user)
ji_bias = beta_items(p_item)

In [58]:
x_ij = ui_bias + ji_bias + ui_latent_factor.mm(vj_hidden_info.T) + ui_content_factor.mm(f_j.T)
x_ij

tensor([[10.7693]], grad_fn=<AddBackward0>)